In [1]:
import os
pm_path = os.path.join(os.getenv('appdata'), 'PigiPlus', 'PropertyModel.json')

In [2]:
# load pm config json into big dictionary

import json
with open(pm_path, 'r', encoding='utf-8') as f:
    pm_entities = json.load(f)

In [3]:
# check it worked

for name, entities in pm_entities.items():
    print(f'{len(entities):>5} {name}')

 3128 Properties
   79 PropertyGroups
  101 PropertyGroupAliases
 9147 PropertyGroupProperties
56321 PropertyGroupPropertyAliases
14650 Equations
    3 IndicatorGroups
   12 Indicators
    2 RatioGroups
    5 Ratios
   36 UnitOfMeasureGroups
  134 UnitsOfMeasure
  462 UnitOfMeasureAliases
34727 MeasurementContexts
   59 Codelists
 2317 CodelistEntries
  371 CodelistEntryAliases


In [7]:
# todo - can't find mappings from GUID yet
#guid_to_data_type: Dict[str, str] = {   # Hard coding as they don't seem to be in the json
  
#}

In [4]:
# types listed in order for dependency graph
t_str = "PropertyGroup, PropertyGroupAlias, IndicatorGroup, Indicator, \
    RatioGroup, Ratio, UnitOfMeasureGroup, UnitOfMeasure, UnitOfMeasureAlias, \
    Codelist, CodelistEntry, CodelistEntryAlias, DataType, Property, \
    PropertyGroupProperty, PropertyGroupPropertyAlias, MeasurementContext, Equation"
types = [t.strip() for t in t_str.split(",")]
types

['PropertyGroup',
 'PropertyGroupAlias',
 'IndicatorGroup',
 'Indicator',
 'RatioGroup',
 'Ratio',
 'UnitOfMeasureGroup',
 'UnitOfMeasure',
 'UnitOfMeasureAlias',
 'Codelist',
 'CodelistEntry',
 'CodelistEntryAlias',
 'DataType',
 'Property',
 'PropertyGroupProperty',
 'PropertyGroupPropertyAlias',
 'MeasurementContext',
 'Equation']

In [22]:
def to_python_name(name: str, sep="_") -> str:
    f = lambda i, l: f"{'_' if i != 0 and l.isupper() else ''}{l.lower()}"
    return "".join(f(i, l) for i, l in enumerate(name))

def get_field_name(name: str, sep="_"):
    # replace _id as we will link entities not ids
    return to_python_name(name, sep).replace("_id", "").replace("_value_descriptor", "")

# test
names = ["PropertyGroupAlias", "UnitOfMeasureId"]
for n in names:
    print(f"{n} -> {get_field_name(n)}")

PropertyGroupAlias -> property_group_alias
UnitOfMeasureId -> unit_of_measure


In [24]:
## gen code to create classes
import inflect  # for singlular / plural conv
from typing import Any

p = inflect.engine()

def get_type(entity_key: str, field_key: str) -> str:    
    # handle id types - we will link to entity not guid
    if field_key.endswith("Id"):
        t = field_key.replace("Id", "").replace("Default", "").replace("ValueDescriptor", "")
    else:        
        # can't just look at first, some are none type...
        entities = pm_entities[entity_key]
        dist_types = set(type(e[field_key]).__name__ for e in entities)
        if 'NoneType' in dist_types and len(dist_types) > 1:
            dist_types.remove('NoneType')
        if len(dist_types) > 1:
            print(f"Expected 1 type for {entity_key}.{field_key} (excl None). "
                  f"Got: {dist_types}")
        t = next(iter(dist_types))
    if t.strip() == "":
        return "Any  # Failed to determine type"
    return t
    
def dict_element_to_field(field_key: str, entity_key: str, cls_name: str) -> str:
    if field_key == f"{cls_name}Id":
        return "id: str"
    field_type = get_type(entity_key, field_key)
    return f"{get_field_name(field_key)}: {field_type}"

def get_entity_key(cls_name: str):
    entity_key = p.plural_noun(cls_name)
    # fix plural conv mistakes!
    entity_key = entity_key.replace('Aliass', 'Aliases')
    entity_key = entity_key.replace('UnitOfMeasures', 'UnitsOfMeasure')
    entity_key = entity_key.replace('ys', 'ies')
    return entity_key

# using types defined above as this has the order I want
for cls_name in types:
    try:
        entity_key = get_entity_key(cls_name)
        field_keys = pm_entities[entity_key][0].keys()
        fields = [dict_element_to_field(field_key, entity_key, cls_name)
                  for field_key in field_keys]
            
        fields_str = "\n    ".join(fields)

        cl = f"""
@dataclass(frozen=True)
class {cls_name}:
    {fields_str}
        """
        print(cl)
    except Exception as e:
        print(f"# failed to autogen class for {cls_name}:\n# {repr(e)}\n")


@dataclass(frozen=True)
class PropertyGroup:
    id: str
    short_name: str
    long_name: str
    igi_code: str
    is_analysis: bool
    list_position: int
        

@dataclass(frozen=True)
class PropertyGroupAlias:
    id: str
    property_group: PropertyGroup
    alias: str
        

@dataclass(frozen=True)
class IndicatorGroup:
    id: str
    name: str
        

@dataclass(frozen=True)
class Indicator:
    id: str
    indicator_group: IndicatorGroup
    long_name: str
    short_name: str
    igi_code: str
    is_default: bool
    is_null_equivalent: bool
    position_in_indicator_group: int
        

@dataclass(frozen=True)
class RatioGroup:
    id: str
    name: str
        

@dataclass(frozen=True)
class Ratio:
    id: str
    ratio_group: RatioGroup
    long_name: str
    symbol: str
    igi_code: str
    is_base: bool
    is_null_equivalent: bool
    position_in_ratio_group: int
    conversion_parameters: dict
        

@dataclass(frozen=True)
class UnitOfMeasureGroup:
    

In [9]:
# gen code to populate objects from json

for t in types:
    template = f"""
    def get_{to_python_name(t)}_lookup(self, 
        config: List[Dict[str, Any]]) -> Dict[str, {t}]:
        lookup = {{}}
        for c in config:
            guid = c['{t}Id']
            lookup[guid] = {t}(
                id=c[guid],
                
                )
        return lookup"""
    print(template)


    def get_property_group_lookup(self, 
        config: List[Dict[str, Any]]) -> Dict[str, PropertyGroup]:
        lookup = {}
        for c in config:
            guid = c['PropertyGroupId']
            lookup[guid] = PropertyGroup(
                id=c[guid],
                
                )
        return lookup

    def get_property_group_alias_lookup(self, 
        config: List[Dict[str, Any]]) -> Dict[str, PropertyGroupAlias]:
        lookup = {}
        for c in config:
            guid = c['PropertyGroupAliasId']
            lookup[guid] = PropertyGroupAlias(
                id=c[guid],
                
                )
        return lookup

    def get_indicator_group_lookup(self, 
        config: List[Dict[str, Any]]) -> Dict[str, IndicatorGroup]:
        lookup = {}
        for c in config:
            guid = c['IndicatorGroupId']
            lookup[guid] = IndicatorGroup(
                id=c[guid],
                
                )
        return lookup

    def get